In [160]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torchtext
import pandas as pd
import os
from string import punctuation
from nltk import word_tokenize
from torchtext.vocab import build_vocab_from_iterator
import math
from sklearn.model_selection import train_test_split
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from tqdm import tqdm
import models
import data_reader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
data_vie = data_reader.read_text("./eng_vie/vi_sents")
data_eng = data_reader.read_text("./eng_vie/en_sents")

sents_vie = data_reader.to_lines(data_vie)
sents_eng = data_reader.to_lines(data_eng)

In [5]:
dataset = pd.DataFrame({'sents_eng': sents_eng, 'sents_vie': sents_vie})

In [6]:
dataset.head()

,sents_eng,sents_vie
0,Please put the dustpan in the broom closet,xin vui lòng đặt người quét rác trong tủ chổi
1,Be quiet for a moment.,im lặng một lát
2,Read this,đọc này
3,Tom persuaded the store manager to give him ba...,tom thuyết phục người quản lý cửa hàng trả lại...
4,Friendship consists of mutual understanding,tình bạn bao gồm sự hiểu biết lẫn nhau


In [7]:
def clean_text(text, lowercase=False, remove_punc=False, remove_num=False, sos_token='<sos>', eos_token='<eos>'):
    if lowercase:
        text = text.lower()
    if remove_punc:
        text = ''.join([ch for ch in text if ch not in punctuation])
    if remove_num:
        text = ''.join([ch for ch in text if ch not in '1234567890'])
    text = [sos_token] + word_tokenize(text) + [eos_token]
    return text

In [8]:
dataset['clean_eng'] = dataset['sents_eng'].apply(lambda x: clean_text(x, lowercase=True, remove_punc=True, remove_num=False))
dataset['clean_vie'] = dataset['sents_vie'].apply(lambda x: clean_text(x, lowercase=True, remove_punc=True, remove_num=False))

In [9]:
dataset.head()

,sents_eng,sents_vie,clean_eng,clean_vie
0,Please put the dustpan in the broom closet,xin vui lòng đặt người quét rác trong tủ chổi,"[<sos>, please, put, the, dustpan, in, the, br...","[<sos>, xin, vui, lòng, đặt, người, quét, rác,..."
1,Be quiet for a moment.,im lặng một lát,"[<sos>, be, quiet, for, a, moment, <eos>]","[<sos>, im, lặng, một, lát, <eos>]"
2,Read this,đọc này,"[<sos>, read, this, <eos>]","[<sos>, đọc, này, <eos>]"
3,Tom persuaded the store manager to give him ba...,tom thuyết phục người quản lý cửa hàng trả lại...,"[<sos>, tom, persuaded, the, store, manager, t...","[<sos>, tom, thuyết, phục, người, quản, lý, cử..."
4,Friendship consists of mutual understanding,tình bạn bao gồm sự hiểu biết lẫn nhau,"[<sos>, friendship, consists, of, mutual, unde...","[<sos>, tình, bạn, bao, gồm, sự, hiểu, biết, l..."


In [10]:
unk_token = '<unk>'
pad_token = '<pad>'
sos_token = '<sos>'
eos_token = '<eos>'

In [11]:
specials = [unk_token, pad_token, sos_token, eos_token]

In [12]:
eng_vocab = build_vocab_from_iterator(dataset['clean_eng'], specials = specials)
vie_vocab = build_vocab_from_iterator(dataset['clean_vie'], specials = specials)

In [13]:
vie_vocab.get_itos()[:10], eng_vocab.get_itos()[:10]

(['<unk>',
  '<pad>',
  '<sos>',
  '<eos>',
  'tôi',
  'bạn',
  'không',
  'tom',
  'có',
  'một'],
 ['<unk>', '<pad>', '<sos>', '<eos>', 'the', 'to', 'i', 'tom', 'you', 'a'])

In [14]:
def text_to_number(text, vocab):
    return vocab.lookup_indices(text)

In [15]:
dataset['eng_nums'] = dataset['clean_eng'].apply(lambda x: text_to_number(x, eng_vocab))
dataset['vie_nums'] = dataset['clean_vie'].apply(lambda x: text_to_number(x, vie_vocab))

In [16]:
dataset.head()

,sents_eng,sents_vie,clean_eng,clean_vie,eng_nums,vie_nums
0,Please put the dustpan in the broom closet,xin vui lòng đặt người quét rác trong tủ chổi,"[<sos>, please, put, the, dustpan, in, the, br...","[<sos>, xin, vui, lòng, đặt, người, quét, rác,...","[2, 104, 145, 4, 15795, 12, 4, 5736, 2020, 3]","[2, 161, 120, 173, 239, 33, 1742, 1282, 31, 92..."
1,Be quiet for a moment.,im lặng một lát,"[<sos>, be, quiet, for, a, moment, <eos>]","[<sos>, im, lặng, một, lát, <eos>]","[2, 25, 729, 20, 9, 495, 3]","[2, 668, 606, 9, 1121, 3]"
2,Read this,đọc này,"[<sos>, read, this, <eos>]","[<sos>, đọc, này, <eos>]","[2, 244, 19, 3]","[2, 235, 30, 3]"
3,Tom persuaded the store manager to give him ba...,tom thuyết phục người quản lý cửa hàng trả lại...,"[<sos>, tom, persuaded, the, store, manager, t...","[<sos>, tom, thuyết, phục, người, quản, lý, cử...","[2, 7, 1982, 4, 576, 1515, 5, 122, 43, 109, 24...","[2, 7, 451, 426, 33, 777, 212, 158, 148, 178, ..."
4,Friendship consists of mutual understanding,tình bạn bao gồm sự hiểu biết lẫn nhau,"[<sos>, friendship, consists, of, mutual, unde...","[<sos>, tình, bạn, bao, gồm, sự, hiểu, biết, l...","[2, 2027, 3004, 13, 5537, 2324, 3]","[2, 312, 5, 71, 1257, 57, 211, 37, 1187, 156, 3]"


In [17]:
def train_test(dataset, test_size = 0.2):
    train, test = train_test_split(dataset, test_size=test_size, random_state = 42)
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    return train, test

In [18]:
train, test = train_test(dataset)

In [19]:
train.head()

,sents_eng,sents_vie,clean_eng,clean_vie,eng_nums,vie_nums
0,He was never to see his homeland again,anh không bao giờ được gặp lại quê hương,"[<sos>, he, was, never, to, see, his, homeland...","[<sos>, anh, không, bao, giờ, được, gặp, lại, ...","[2, 11, 15, 97, 5, 83, 24, 8738, 186, 3]","[2, 11, 6, 71, 49, 41, 103, 62, 1424, 1461, 3]"
1,She had a basket full of apples,cô ấy có một giỏ đầy táo,"[<sos>, she, had, a, basket, full, of, apples,...","[<sos>, cô, ấy, có, một, giỏ, đầy, táo, <eos>]","[2, 35, 48, 9, 2274, 644, 13, 1085, 3]","[2, 26, 14, 8, 9, 1736, 594, 701, 3]"
2,There was a larger crowd at the concert than w...,có một đám đông lớn hơn tại buổi hòa nhạc hơn ...,"[<sos>, there, was, a, larger, crowd, at, the,...","[<sos>, có, một, đám, đông, lớn, hơn, tại, buổ...","[2, 56, 15, 9, 1919, 1093, 37, 4, 793, 103, 23...","[2, 8, 9, 649, 551, 160, 66, 80, 363, 501, 355..."
3,"Windy this morning, isn't it?",sáng nay có gió không?,"[<sos>, windy, this, morning, isnt, it, <eos>]","[<sos>, sáng, nay, có, gió, không, <eos>]","[2, 3844, 19, 237, 150, 16, 3]","[2, 210, 112, 8, 1032, 6, 3]"
4,I worked hard for this money.,tôi đã làm việc chăm chỉ vì số tiền này,"[<sos>, i, worked, hard, for, this, money, <eos>]","[<sos>, tôi, đã, làm, việc, chăm, chỉ, vì, số,...","[2, 6, 619, 219, 20, 19, 116, 3]","[2, 4, 10, 18, 59, 324, 75, 104, 118, 115, 30, 3]"


In [20]:
pad_index = eng_vocab[pad_token]
unk_index = eng_vocab[unk_token]

In [21]:
vie_vocab.set_default_index(unk_index)
eng_vocab.set_default_index(unk_index)

In [22]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        eng_nums, viet_nums = torch.tensor(self.data.iloc[idx]['eng_nums']), torch.tensor(self.data.iloc[idx]['vie_nums'])
        return eng_nums, viet_nums


In [23]:
train_dataset = CustomDataset(train)
test_dataset = CustomDataset(test)

In [24]:
train_dataset[0], test_dataset[0]

((tensor([   2,   11,   15,   97,    5,   83,   24, 8738,  186,    3]),
  tensor([   2,   11,    6,   71,   49,   41,  103,   62, 1424, 1461,    3])),
 (tensor([  2,   6, 282,  22, 737,   4, 173, 123,   3]),
  tensor([  2,   4,  10, 186, 201, 300,  12,   4,  45,  64, 127,   3])))

In [25]:
batch_size = 512
num_workers = 4

In [26]:
def collate_fn(batch):
    eng_numerical = [item[0] for item in batch]
    vie_numerical = [item[1] for item in batch]
    
    eng_numerical = nn.utils.rnn.pad_sequence(eng_numerical, padding_value=pad_index)
    vie_numerical = nn.utils.rnn.pad_sequence(vie_numerical, padding_value=pad_index)
    
    return eng_numerical, vie_numerical

In [27]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn= collate_fn, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers,collate_fn=collate_fn, shuffle=False)

In [88]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == pad_index).transpose(0, 1)
    tgt_padding_mask = (tgt == pad_index).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [161]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(eng_vocab)
TGT_VOCAB_SIZE = len(vie_vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = models.EngToVieTranslation(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

transformer = transformer.to(device)


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [33]:
def train_fn(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for question, answer in data_loader:
        question = question.to(device)
        answer = answer.to(device)
        
        answer_in = answer[:-1, :]
        
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(question, answer_in)
                
        
        output = model(question, answer_in, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask).to(device)
        
        optimizer.zero_grad()
        
        answer_out = answer[1:, :]

        output = output.view(-1, output.size(-1))
        answer_out = answer_out.view(-1)
        
        loss = criterion(output, answer_out)
        
        loss.backward()
        
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(data_loader)

In [34]:
def eval_fn(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for question, answer in data_loader:
            question = question.to(device)
            answer = answer.to(device)

            answer_in = answer[:-1, :]

            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(question, answer_in)

            output = model(question, answer_in, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask).to(device)

            answer_out = answer[1:, :]

            output = output.view(-1, output.size(-1))
            answer_out = answer_out.view(-1)

            loss = criterion(output, answer_out)

            total_loss += loss.item()
        
    return total_loss / len(data_loader)

In [35]:
epochs = 100
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=pad_index)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001)

In [36]:
min_loss = float('inf')
for epoch in tqdm(range(epochs)):
    train_loss = train_fn(transformer, train_loader, loss_fn, optimizer, device)
    test_loss = eval_fn(transformer, test_loader, loss_fn, device)
    
    if test_loss < min_loss:
        min_loss = test_loss
        torch.save(transformer.state_dict(), "best_translate_model.pth")
        
    print(f"Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss}")
torch.save(transformer.state_dict(), "last_translate_model.pth")

  0%|          | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
  1%|          | 1/100 [02:04<3:24:49, 124.13s/it]

Epoch: 0, Train Loss: 4.677931672364623, Test Loss: 3.493857219219208


  2%|▏         | 2/100 [04:08<3:23:03, 124.32s/it]

Epoch: 1, Train Loss: 3.228267881139439, Test Loss: 2.7307542824745177


  3%|▎         | 3/100 [06:12<3:20:41, 124.14s/it]

...

Epoch: 97, Train Loss: 0.2116531738683806, Test Loss: 1.0546039605140687


 99%|█████████▉| 99/100 [3:24:11<02:03, 123.74s/it]

Epoch: 98, Train Loss: 0.21071938515158753, Test Loss: 1.0658611744642257


100%|██████████| 100/100 [3:26:15<00:00, 123.75s/it]

Epoch: 99, Train Loss: 0.2113868618505684, Test Loss: 1.064922543168068
